In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from math import sqrt
from ipywidgets import IntProgress
from IPython.display import display
from ipywidgets import FileUpload
from IPython.display import display, clear_output
from ipywidgets import BoundedFloatText, widgets, VBox, HBox
from plotly.subplots import make_subplots
import plotly.express as px
import numpy as np
import pandas as pd
from pycaret.regression import *
import os
import dash_bio as dashbio

## Proffinity visualization module

In [2]:
content={"input":"","fi":"","outname":""}

In [3]:
out = widgets.Output(layout={'border': '1px solid black','width':'1000px'})
#out

In [4]:
#fw=pd.DataFrame()
def load_ipywiget_model(widget):
    content['fi'] = list(widget['new'])[0]
    fw=pd.read_csv(content['fi'])
    fig_bar = px.bar(fw, x='feature', y='weight',width=900, height=400)
    with out:
        clear_output()
        display("load "+content['fi'])
        display(fig_bar)

In [5]:
def on_button_clicked(b):
    path = './'
    dropdown = widgets.SelectMultiple(
                        options=[f for f in os.listdir(path) if f.endswith('.csv') and f.startswith('feature_importance')],
                        description='models',
                        disabled=False,
                        layout={'height':'100px', 'width':'40%'})
    dropdown.observe(load_ipywiget_model, names='value')
    

    with out:
        clear_output()
        display(dropdown)

In [6]:
button = widgets.Button(description="Load FI params")
button.on_click(on_button_clicked)
#button

In [7]:
out2 = widgets.Output(layout={'border': '1px solid black','width':'1000px'})
#out2

In [8]:
def load_input(widget):
    content['input'] = list(widget['new'])[0]
    content['outname'] = list(widget['new'])[0].split('.')[0]
    df = pd.read_csv(content["input"], header=0, index_col=0)
    
    with out2:
        display("load "+content['input'])
        display(df)

In [9]:
def on_button2_clicked(b):
    path = './model/'
    dropdown = widgets.SelectMultiple(
                        options=[f for f in os.listdir(path) if f.endswith('.pdb')],
                        description='inputs',
                        disabled=False,
                        layout={'height':'100px', 'width':'40%'})
    dropdown.observe(load_input, names='value')

    with out2:
        clear_output()
        display(dropdown)

In [10]:
button2 = widgets.Button(description="Load Inputs")
button2.on_click(on_button2_clicked)
#button2

In [11]:
#content

In [12]:
out3 = widgets.Output(layout={'border': '1px solid black','width':'1000px'})
#out3

In [13]:
def cb_score():
    pdb=content['outname']
    fw=pd.read_csv(content['fi'])
    df_cb=pd.read_csv("./raw_graphv2/"+pdb+"_CB_output.txt", header=None)
    df_cb['index1']=df_cb.iloc[:,0]+"_"+df_cb.iloc[:,1]+"_"+df_cb.iloc[:,2].astype(str)
    df_cb_score=df_cb.iloc[:,13:].groupby(df_cb['index1'], sort=False).sum()
    df_cb['index2']=df_cb.iloc[:,3]+"_"+df_cb.iloc[:,4]+"_"+df_cb.iloc[:,5].astype(str)
    df_cb_binary_score=(df_cb[~df_cb.filter(like='index').apply(frozenset, axis=1).duplicated()].reset_index(drop=True).iloc[:,13:])

    scoreSB_CB_w=fw[fw["feature"]=='scoreSB_CB']['weight']
    scoreMDS_CB_w=fw[fw["feature"]=='scoreMDS_CB']['weight']
    scoreMDW_CB_w=fw[fw["feature"]=='scoreMDW_CB']['weight']
    scoreKP_CB_w=fw[fw["feature"]=='scoreKP_CB']['weight']
    scoreRRP_CB_w=fw[fw["feature"]=='scoreRRP_CB']['weight']
    scoreMNJ_Etot_CB_w=fw[fw["feature"]=='scoreMNJ_Etot_CB']['weight']
    scoreMNJ_Elj_CB_w=fw[fw["feature"]=='scoreMNJ_Elj_CB']['weight']
    scoreMNJ_Ee_CB_w=fw[fw["feature"]=='scoreMNJ_Ee_CB']['weight']
    scoreMNJ_Ehb_CB_w=fw[fw["feature"]=='scoreMNJ_Ehb_CB']['weight']
    scoreEE_CB_w=fw[fw["feature"]=='scoreEE_CB']['weight']
    scoreEC_CB_w=fw[fw["feature"]=='scoreEC_CB']['weight']
    
    weight_cb=np.array([scoreSB_CB_w, 
                    scoreMDS_CB_w, 
                    scoreMDW_CB_w, 
                    scoreKP_CB_w, 
                    scoreRRP_CB_w, 
                    scoreMNJ_Etot_CB_w, 
                    scoreMNJ_Elj_CB_w,
                    scoreMNJ_Ee_CB_w,
                    scoreMNJ_Ehb_CB_w,
                    scoreEE_CB_w,
                    scoreEC_CB_w])
    df_cb_score['weight_score']=df_cb_score.iloc[:,0:11].dot(weight_cb)
    df_cb_binary_score['weight_score']=df_cb_binary_score.iloc[:,0:11].dot(weight_cb)
    return df_cb_score, df_cb_binary_score
    

In [14]:
def ca_score():
    pdb=content['outname']
    fw=pd.read_csv(content['fi'])
    df_ca=pd.read_csv("./raw_graphv2/"+pdb+"_CA_output.txt", header=None)
    df_ca['index1']=df_ca.iloc[:,0]+"_"+df_ca.iloc[:,1]+"_"+df_ca.iloc[:,2].astype(str)
    df_ca_score=df_ca.iloc[:,13:].groupby(df_ca['index1'], sort=False).sum()
    df_ca['index2']=df_ca.iloc[:,3]+"_"+df_ca.iloc[:,4]+"_"+df_ca.iloc[:,5].astype(str)
    df_ca_binary_score=(df_ca[~df_ca.filter(like='index').apply(frozenset, axis=1).duplicated()].reset_index(drop=True).iloc[:,13:])
    
    scoreSB_CA_w=fw[fw["feature"]=='scoreSB_CA']['weight']
    scoreMDS_CA_w=fw[fw["feature"]=='scoreMDS_CA']['weight']
    scoreMDW_CA_w=fw[fw["feature"]=='scoreMDW_CA']['weight']
    scoreKP_CA_w=fw[fw["feature"]=='scoreKP_CA']['weight']
    scoreRRP_CA_w=fw[fw["feature"]=='scoreRRP_CA']['weight']
    scoreMNJ_Etot_CA_w=fw[fw["feature"]=='scoreMNJ_Etot_CA']['weight']
    scoreMNJ_Elj_CA_w=fw[fw["feature"]=='scoreMNJ_Elj_CA']['weight']
    scoreMNJ_Ee_CA_w=fw[fw["feature"]=='scoreMNJ_Ee_CA']['weight']
    scoreMNJ_Ehb_CA_w=fw[fw["feature"]=='scoreMNJ_Ehb_CA']['weight']
    scoreEE_CA_w=fw[fw["feature"]=='scoreEE_CA']['weight']
    scoreEC_CA_w=fw[fw["feature"]=='scoreEC_CA']['weight']
    
    weight_ca=np.array([scoreSB_CA_w, 
                    scoreMDS_CA_w, 
                    scoreMDW_CA_w, 
                    scoreKP_CA_w, 
                    scoreRRP_CA_w, 
                    scoreMNJ_Etot_CA_w, 
                    scoreMNJ_Elj_CA_w,
                    scoreMNJ_Ee_CA_w,
                    scoreMNJ_Ehb_CA_w,
                    scoreEE_CA_w,
                    scoreEC_CA_w])
    df_ca_score['weight_score']=df_ca_score.iloc[:,0:11].dot(weight_ca)
    df_ca_binary_score['weight_score']=df_ca_binary_score.iloc[:,0:11].dot(weight_ca)
    return df_ca_score, df_ca_binary_score
    

In [15]:
def cb_bb_nb_score():
    pdb=content['outname']
    fw=pd.read_csv(content['fi'])
    df_cb_bb_nb=pd.read_csv("./raw_graphv2/"+pdb+"_CB_output_bb.txt", header=None)
    df_cb_bb_nb['index1']=df_cb_bb_nb.iloc[:,0]+"_"+df_cb_bb_nb.iloc[:,1]+"_"+df_cb_bb_nb.iloc[:,2].astype(str)
    df_cb_bb_nb_score=df_cb_bb_nb.iloc[:,13:].groupby(df_cb_bb_nb['index1'], sort=False).sum()
    df_cb_bb_nb['index2']=df_cb_bb_nb.iloc[:,3]+"_"+df_cb_bb_nb.iloc[:,4]+"_"+df_cb_bb_nb.iloc[:,5].astype(str)
    df_cb_bb_nb_binary_score=(df_cb_bb_nb[~df_cb_bb_nb.filter(like='index').apply(frozenset, axis=1).duplicated()].reset_index(drop=True).iloc[:,13:])
    
    scoreSB_BB_NB_CB_w=fw[fw["feature"]=='scoreSB_BB_NB_CB']['weight']
    scoreMDS_BB_NB_CB_w=fw[fw["feature"]=='scoreMDS_BB_NB_CB']['weight']
    scoreMDW_BB_NB_CB_w=fw[fw["feature"]=='scoreMDW_BB_NB_CB']['weight']
    scoreKP_BB_NB_CB_w=fw[fw["feature"]=='scoreKP_BB_NB_CB']['weight']
    scoreRRP_BB_NB_CB_w=fw[fw["feature"]=='scoreRRP_BB_NB_CB']['weight']
    scoreMNJ_BB_NB_Etot_CB_w=fw[fw["feature"]=='scoreMNJ_BB_NB_Etot_CB']['weight']
    scoreMNJ_BB_NB_Elj_CB_w=fw[fw["feature"]=='scoreMNJ_BB_NB_Elj_CB']['weight']
    scoreMNJ_BB_NB_Ee_CB_w=fw[fw["feature"]=='scoreMNJ_BB_NB_Ee_CB']['weight']
    scoreMNJ_BB_NB_Ehb_CB_w=fw[fw["feature"]=='scoreMNJ_BB_NB_Ehb_CB']['weight']
    scoreEE_BB_NB_CB_w=fw[fw["feature"]=='scoreEE_BB_NB_CB']['weight']
    scoreEC_BB_NB_CB_w=fw[fw["feature"]=='scoreEC_BB_NB_CB']['weight']
    
    weight_cb_bb_nb=np.array([scoreSB_BB_NB_CB_w, 
                    scoreMDS_BB_NB_CB_w, 
                    scoreMDW_BB_NB_CB_w, 
                    scoreKP_BB_NB_CB_w, 
                    scoreRRP_BB_NB_CB_w, 
                    scoreMNJ_BB_NB_Etot_CB_w, 
                    scoreMNJ_BB_NB_Elj_CB_w,
                    scoreMNJ_BB_NB_Ee_CB_w,
                    scoreMNJ_BB_NB_Ehb_CB_w,
                    scoreEE_BB_NB_CB_w,
                    scoreEC_BB_NB_CB_w])
    
    df_cb_bb_nb_score['weight_score']=df_cb_bb_nb_score.iloc[:,0:11].dot(weight_cb_bb_nb)
    df_cb_bb_nb_binary_score['weight_score']=df_cb_bb_nb_binary_score.iloc[:,0:11].dot(weight_cb_bb_nb)

    return df_cb_bb_nb_score, df_cb_bb_nb_binary_score
    

In [16]:
def ca_bb_nb_score():
    pdb=content['outname']
    fw=pd.read_csv(content['fi'])
    df_ca_bb_nb=pd.read_csv("./raw_graphv2/"+pdb+"_CA_output_bb_nb.txt", header=None)
    df_ca_bb_nb['index1']=df_ca_bb_nb.iloc[:,0]+"_"+df_ca_bb_nb.iloc[:,1]+"_"+df_ca_bb_nb.iloc[:,2].astype(str)
    df_ca_bb_nb_score=df_ca_bb_nb.iloc[:,13:].groupby(df_ca_bb_nb['index1'], sort=False).sum()
    df_ca_bb_nb['index2']=df_ca_bb_nb.iloc[:,3]+"_"+df_ca_bb_nb.iloc[:,4]+"_"+df_ca_bb_nb.iloc[:,5].astype(str)
    df_ca_bb_nb_binary_score=(df_ca_bb_nb[~df_ca_bb_nb.filter(like='index').apply(frozenset, axis=1).duplicated()].reset_index(drop=True).iloc[:,13:])
    
    scoreSB_BB_NB_CA_w=fw[fw["feature"]=='scoreSB_BB_NB_CA']['weight']
    scoreMDS_BB_NB_CA_w=fw[fw["feature"]=='scoreMDS_BB_NB_CA']['weight']
    scoreMDW_BB_NB_CA_w=fw[fw["feature"]=='scoreMDW_BB_NB_CA']['weight']
    scoreKP_BB_NB_CA_w=fw[fw["feature"]=='scoreKP_BB_NB_CA']['weight']
    scoreRRP_BB_NB_CA_w=fw[fw["feature"]=='scoreRRP_BB_NB_CA']['weight']
    scoreMNJ_BB_NB_Etot_CA_w=fw[fw["feature"]=='scoreMNJ_BB_NB_Etot_CA']['weight']
    scoreMNJ_BB_NB_Elj_CA_w=fw[fw["feature"]=='scoreMNJ_BB_NB_Elj_CA']['weight']
    scoreMNJ_BB_NB_Ee_CA_w=fw[fw["feature"]=='scoreMNJ_BB_NB_Ee_CA']['weight']
    scoreMNJ_BB_NB_Ehb_CA_w=fw[fw["feature"]=='scoreMNJ_BB_NB_Ehb_CA']['weight']
    scoreEE_BB_NB_CA_w=fw[fw["feature"]=='scoreEE_BB_NB_CA']['weight']
    scoreEC_BB_NB_CA_w=fw[fw["feature"]=='scoreEC_BB_NB_CA']['weight']
    
    weight_ca_bb_nb=np.array([scoreSB_BB_NB_CA_w, 
                    scoreMDS_BB_NB_CA_w, 
                    scoreMDW_BB_NB_CA_w, 
                    scoreKP_BB_NB_CA_w, 
                    scoreRRP_BB_NB_CA_w, 
                    scoreMNJ_BB_NB_Etot_CA_w, 
                    scoreMNJ_BB_NB_Elj_CA_w,
                    scoreMNJ_BB_NB_Ee_CA_w,
                    scoreMNJ_BB_NB_Ehb_CA_w,
                    scoreEE_BB_NB_CA_w,
                    scoreEC_BB_NB_CA_w])
    
    df_ca_bb_nb_score['weight_score']=df_ca_bb_nb_score.iloc[:,0:11].dot(weight_ca_bb_nb)
    df_ca_bb_nb_binary_score['weight_score']=df_ca_bb_nb_binary_score.iloc[:,0:11].dot(weight_ca_bb_nb)

    return df_ca_bb_nb_score, df_ca_bb_nb_binary_score
    

In [17]:
def ca_bb_b_score():
    pdb=content['outname']
    fw=pd.read_csv(content['fi'])
    df_ca_bb_b=pd.read_csv("./raw_graphv2/"+pdb+"_CA_output_bb_b.txt", header=None)
    df_ca_bb_b['index1']=df_ca_bb_b.iloc[:,0]+"_"+df_ca_bb_b.iloc[:,1]+"_"+df_ca_bb_b.iloc[:,2].astype(str)
    df_ca_bb_b_score=df_ca_bb_b.iloc[:,13:].groupby(df_ca_bb_b['index1'], sort=False).sum()
    scoreSB_BB_Bt_CA_w=fw[fw["feature"]=='scoreSB_BB_Bt_CA']['weight']
    scoreSB_BB_B_CA_w=fw[fw["feature"]=='scoreSB_BB_B_CA']['weight']
    scoreRRP_BB_B_CA_w=fw[fw["feature"]=='scoreRRP_BB_B_CA']['weight']
    weight_ca_bb_b=np.array([scoreSB_BB_Bt_CA_w, scoreSB_BB_B_CA_w, scoreRRP_BB_B_CA_w])
    
    df_ca_bb_b_score['weight_score']=df_ca_bb_b_score.iloc[:,0:3].dot(weight_ca_bb_b)
    df_ca_bb_b_score=df_ca_bb_b_score.drop(columns=['index1'])
    df_ca_bb_b_score.columns=["scoreSB_BB_Bt",
                           "scoreSB_BB_B",
                           "scoreRRP_BB_B",
                          "weight_score_BB_B"
                         ]
    return df_ca_bb_b_score
    

In [18]:
def combine_score(ca_score,cb_score):
    df_combined_score=pd.concat([cb_score, ca_score])
    df_combined_score=df_combined_score.groupby(df_combined_score.index, sort=False).sum()
    df_combined_score=df_combined_score.drop(columns=['index1'])
    df_combined_score.columns=["scoreSB",
                           "scoreMDS",
                           "scoreMDW",
                           "scoreKP",
                           "scoreRRP",
                           "scoreMNJ_Etot",
                           "scoreMNJ_Elj",
                           "scoreMNJ_Ee",
                           "scoreMNJ_Ehb",
                           "scoreEE",
                           "scoreEC",
                           "weight_score"]

    return df_combined_score
    

In [19]:
def bb_nb_combine_score(ca_score,cb_score):
    df_combined_score=pd.concat([cb_score, ca_score])
    df_combined_score=df_combined_score.groupby(df_combined_score.index, sort=False).sum()
    df_combined_score=df_combined_score.drop(columns=['index1'])
    df_combined_score.columns=["scoreSB_BB_NB",
                           "scoreMDS_BB_NB",
                           "scoreMDW_BB_NB",
                           "scoreKP_BB_NB",
                           "scoreRRP_BB_NB",
                           "scoreMNJ_Etot_BB_NB",
                           "scoreMNJ_Elj_BB_NB",
                           "scoreMNJ_Ee_BB_NB",
                           "scoreMNJ_Ehb_BB_NB",
                           "scoreEE_BB_NB",
                           "scoreEC_BB_NB",
                           "weight_score_BB_NB"]

    return df_combined_score

In [20]:
def combine_binary_score(ca_binary_score, cb_binary_score):
    df_binary_combined_score=pd.concat([cb_binary_score, ca_binary_score])
    df_binary_combined_score['Pair'] = df_binary_combined_score.apply(lambda row: tuple(sorted([row['index1'], row['index2']])), axis=1)
    df_binary_combined_score = df_binary_combined_score.groupby('Pair').sum()
    df_binary_combined_score=df_binary_combined_score.drop(columns=['index1','index2'])
    df_binary_combined_score.columns=["scoreSB",
                                  "scoreMDS",
                                  "scoreMDW",
                                  "scoreKP",
                                  "scoreRRP",
                                  "scoreMNJ_Etot",
                                  "scoreMNJ_Elj",
                                  "scoreMNJ_Ee",
                                  "scoreMNJ_Ehb",
                                  "scoreEE",
                                  "scoreEC",
                                  "weight_score"]

    return df_binary_combined_score

In [21]:
def bb_nb_combine_binary_score(ca_binary_score, cb_binary_score):
    df_binary_combined_score=pd.concat([cb_binary_score, ca_binary_score])
    df_binary_combined_score['Pair'] = df_binary_combined_score.apply(lambda row: tuple(sorted([row['index1'], row['index2']])), axis=1)
    df_binary_combined_score = df_binary_combined_score.groupby('Pair').sum()
    df_binary_combined_score=df_binary_combined_score.drop(columns=['index1','index2'])
    df_binary_combined_score.columns=["scoreSB_BB_NB",
                                  "scoreMDS_BB_NB",
                                  "scoreMDW_BB_NB",
                                  "scoreKP_BB_NB",
                                  "scoreRRP_BB_NB",
                                  "scoreMNJ_Etot_BB_NB",
                                  "scoreMNJ_Elj_BB_NB",
                                  "scoreMNJ_Ee_BB_NB",
                                  "scoreMNJ_Ehb_BB_NB",
                                  "scoreEE_BB_NB",
                                  "scoreEC_BB_MB",
                                  "weight_score_BB_NB"]

    return df_binary_combined_score

In [22]:
def all_score(combined_score, bb_nb_combined_score, ca_bb_b_score):
    total_df=pd.merge(ca_bb_b_score, bb_nb_combined_score,on='index1',how='left')
    total_df=pd.merge(total_df, combined_score, on='index1', how='left')
    total_df=total_df.fillna(0)
    total_df['total_weight_score']=total_df['weight_score_BB_B']+total_df['weight_score_BB_NB']+total_df['weight_score']
    return total_df
    

In [23]:
def coord_from_index1():
    pdb=content['outname']
    df_ca_bb_b=pd.read_csv("./raw_graphv2/"+pdb+"_CA_output_bb_b.txt", header=None)
    df_ca_bb_b['index1']=df_ca_bb_b.iloc[:,0]+"_"+df_ca_bb_b.iloc[:,1]+"_"+df_ca_bb_b.iloc[:,2].astype(str)
    df_ca_bb_b=df_ca_bb_b.drop_duplicates(subset=['index1'])
    df_ca_bb_b.index=df_ca_bb_b['index1']
    df_ca_bb_b_cord=df_ca_bb_b.iloc[:,[0,7,8,9,10,11,12]]
    df_ca_bb_b_cord.columns=["chain","x1","y1","z1","x2","y2","z2"]
    return df_ca_bb_b_cord

In [24]:
def combined_binary_score_with_coord(coord, combined_binary_score):
    list_all=[]
    for index, row in combined_binary_score.iterrows():
        cord_x1=coord[coord.index==index[0]]['x1'].values[0]
        cord_y1=coord[coord.index==index[0]]['y1'].values[0]
        cord_z1=coord[coord.index==index[0]]['z1'].values[0]
        cord_x2=coord[coord.index==index[1]]['x1'].values[0]
        cord_y2=coord[coord.index==index[1]]['y1'].values[0]
        cord_z2=coord[coord.index==index[1]]['z1'].values[0]
        weight_score=row['weight_score']
        list=[index[0],index[1],cord_x1,cord_y1,cord_z1,cord_x2,cord_y2,cord_z2,weight_score]
        list_all.append(list)
        
    weighted_edge_score=pd.DataFrame(list_all)
    weighted_edge_score.columns=["index1","index2","x1","y1","z1","x2","y2","z2","edge_weight_score"]
    weighted_edge_score.index=weighted_edge_score.iloc[:,0]+"_"+weighted_edge_score.iloc[:,1]
    weighted_edge_score.index.name='index'
    return weighted_edge_score

In [25]:
def bb_nb_combined_binary_score_with_coord(coord, bb_nb_combined_binary_score):
    list_all=[]
    for index, row in bb_nb_combined_binary_score.iterrows():
        cord_x1=coord[coord.index==index[0]]['x1'].values[0]
        cord_y1=coord[coord.index==index[0]]['y1'].values[0]
        cord_z1=coord[coord.index==index[0]]['z1'].values[0]
        cord_x2=coord[coord.index==index[1]]['x1'].values[0]
        cord_y2=coord[coord.index==index[1]]['y1'].values[0]
        cord_z2=coord[coord.index==index[1]]['z1'].values[0]
        weight_score=row['weight_score_BB_NB']
        list=[index[0],index[1],cord_x1,cord_y1,cord_z1,cord_x2,cord_y2,cord_z2,weight_score]
        list_all.append(list)
        
    bb_nb_weighted_edge_score=pd.DataFrame(list_all)
    bb_nb_weighted_edge_score.columns=["index1","index2","x1","y1","z1","x2","y2","z2","edge_weight_score_BB_NB"]
    bb_nb_weighted_edge_score.index=bb_nb_weighted_edge_score.iloc[:,0]+"_"+bb_nb_weighted_edge_score.iloc[:,1]
    bb_nb_weighted_edge_score.index.name='index'
    return bb_nb_weighted_edge_score

In [26]:
selector = widgets.RadioButtons(
    options=['Weighted residue scores-IF', 'Weighted residue scores-FD', 'Weighted interaction scores-IF', 'Weighted interaction scores-FD'],
    value='Weighted residue scores-IF',
    description='',
    disabled=False
)

In [27]:
selector2 = widgets.RadioButtons(
    options=['label on', 'label off'],
    value='label off',
    description='',
    disabled=False
)

In [28]:
selector3 = widgets.RadioButtons(
    options=['-1/1', '0/1'],
    value='0/1',
    description='',
    disabled=False
)

In [29]:
slider1=widgets.FloatSlider(
    value=-1,
    min=-1,
    max=1,
    step=0.1,
    description='WRS-IF:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.1f',
)

slider2=widgets.FloatSlider(
    value=0,
    min=-1,
    max=1,
    step=0.1,
    description='WRS-FD:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.1f',
)

slider3=widgets.FloatSlider(
    value=0,
    min=-1,
    max=1,
    step=0.1,
    description='WIS-IF:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.1f',
)



slider4=widgets.FloatSlider(
    value=0,
    min=-1,
    max=1,
    step=0.1,
    description='WIS-FD:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.1f',
)

slider5=widgets.FloatSlider(
    value=8,
    min=8,
    max=15,
    step=0.5,
    description='edge width:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.1f',
)

slider6=widgets.FloatSlider(
    value=5,
    min=1,
    max=10,
    step=0.5,
    description='chain width:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.1f',
)

slider7=widgets.FloatSlider(
    value=0.1,
    min=0,
    max=1,
    step=0.1,
    description='edge thres:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.1f',
)



In [30]:
def min_max_scale(df, feature, min, max):
    a = min
    b = max
    x = df[feature]
    df[feature] = a + (x - x.min()) * (b - a) / (x.max() - x.min())
    return df

In [31]:
def plot_structuresv2(coord, bb_nb_combined_score, combined_score, bb_nb_combined_binary_score, combined_binary_score,wes,bb_nb_wes):
    
    if (selector.value == 'Weighted residue scores-IF'):
        total_df=pd.merge(combined_score, coord, on='index1', how='left')
        total_df=total_df[total_df['weight_score'] >= slider1.value]
        avg_score=total_df['weight_score'].mean()
        sum_score=total_df['weight_score'].sum()
        N=total_df.shape[0]
        fig1_1 = px.line_3d(coord, x="x1", y="y1", z="z1",color="chain", width=600, height=600)
        #fig1_1.update_traces(showlegend = False, line=dict(color='chain',width=10))
        fig1_1.update_traces(showlegend = False)
        fig1_1.update_traces(line=dict(width=slider6.value))
        
        
        if (selector2.value == 'label on'):
            fig1_2 = px.scatter_3d(total_df, x="x1", y="y1", z="z1",color="weight_score",width=600, height=600, text="label")
        if (selector2.value == 'label off'):
            fig1_2 = px.scatter_3d(total_df, x="x1", y="y1", z="z1",color="weight_score",width=600, height=600)
        
        #fig1_2.update_traces(marker_size = 3)
        fig1_1.add_trace(fig1_2.data[0])
        total_df=total_df.iloc[:,0:12]
        total_df=(total_df-total_df.min())/(total_df.max()-total_df.min())
        total_df=total_df.fillna(0)
        total_df.T.to_csv("vis_"+content["outname"]+"_by_"+selector.value+".csv")



        #columns = list(total_df.columns.values)
        #rows = list(total_df.index)

        #clustergram = dashbio.Clustergram(
        #    data=total_df,
        #    row_labels=rows,
        #    column_labels=columns,
        #    color_threshold={
        #        'row': 250,
        #        'col': 700
        #    },
        #    height=800,
        #    width=700,
        #    color_map= [
        #        [0.0, '#636EFA'],
        #        [0.25, '#AB63FA'],
        #        [0.5, '#FFFFFF'],
        #        [0.75, '#E763FA'],
        #        [1.0, '#EF553B']
        #    ]    
        #)
        
        #fig1_hmp = px.imshow(total_df.iloc[:,0:12], x=total_df.iloc[:,0:12].columns, y=total_df.iloc[:,0:12].index, aspect='auto')
        fig1_hmp = px.imshow(total_df, x=total_df.columns, y=total_df.index, aspect='auto')
        fig1_hmp.update_layout(width=600,height=600)
        return fig1_1, fig1_hmp, avg_score, sum_score, N

    if (selector.value == 'Weighted residue scores-FD'):
        
        bb_nb_total_df=pd.merge(bb_nb_combined_score, coord, on='index1', how='left')
        bb_nb_total_df=bb_nb_total_df[bb_nb_total_df['weight_score_BB_NB'] >= slider2.value]
        avg_score=bb_nb_total_df['weight_score_BB_NB'].mean()
        sum_score=bb_nb_total_df['weight_score_BB_NB'].sum()
        N=bb_nb_total_df.shape[0]
        fig2_1 = px.line_3d(coord, x="x1", y="y1", z="z1",color="chain",width=600, height=600)
        fig2_1.update_traces(showlegend = False)
        fig2_1.update_traces(line=dict(width=slider6.value))

        if (selector2.value == 'label on'):
            fig2_2 = px.scatter_3d(bb_nb_total_df, x="x1", y="y1", z="z1",color='weight_score_BB_NB',width=600, height=600,text="label")
        if (selector2.value == 'label off'):
            fig2_2 = px.scatter_3d(bb_nb_total_df, x="x1", y="y1", z="z1",color='weight_score_BB_NB',width=600, height=600)
        fig2_1.add_trace(fig2_2.data[0])

        bb_nb_total_df=bb_nb_total_df.iloc[:,0:12]
        bb_nb_total_df=(bb_nb_total_df-bb_nb_total_df.min())/(bb_nb_total_df.max()-bb_nb_total_df.min())
        bb_nb_total_df=bb_nb_total_df.fillna(0)
        bb_nb_total_df.T.to_csv("vis_"+content["outname"]+"_by_"+selector.value+".csv")
        
        fig2_hmp = px.imshow(bb_nb_total_df, x=bb_nb_total_df.columns, y=bb_nb_total_df.index,aspect='auto')
        #fig2_hmp = px.imshow(bb_nb_total_df.iloc[:,0:12], x=bb_nb_total_df.iloc[:,0:12].columns, y=bb_nb_total_df.iloc[:,0:12].index,aspect='auto')
        fig2_hmp.update_layout(width=600,height=600)
       
        return fig2_1, fig2_hmp, avg_score, sum_score, N

    if (selector.value == 'Weighted interaction scores-IF'):
        
        total_df=pd.merge(combined_score, coord, on='index1', how='left')
        total_df=total_df[total_df['weight_score'] >= slider1.value]
        combined_binary_score=combined_binary_score[combined_binary_score['edge_weight_score'] >= slider3.value]
        wes=wes[wes['weight_score'] >= slider3.value]
        avg_score=wes['weight_score'].mean()
        sum_score=wes['weight_score'].sum()
        N=wes.shape[0]
        
        fig3_1 = px.line_3d(coord, x="x1", y="y1", z="z1", color="chain",width=600, height=600)
        fig3_1.update_traces(showlegend = False)
        fig3_1.update_traces(line=dict(width=slider6.value))

        if (selector2.value == 'label on'):
            fig3_2 = px.scatter_3d(total_df, x="x1", y="y1", z="z1",color="weight_score",width=600, height=600, text="label")
            fig3_1.add_trace(fig3_2.data[0])
        if (selector2.value == 'label off'):
            fig3_2 = px.scatter_3d(total_df, x="x1", y="y1", z="z1",color="weight_score",width=600, height=600)
            fig3_1.add_trace(fig3_2.data[0])

        #normalized again after cutoff, use defined threshold on screen based on heatmap
        wes=wes.iloc[:,0:13]
        wes=(wes-wes.min())/(wes.max()-wes.min())
        wes=wes.fillna(0)
        combined_binary_score['edge_weight_score']=wes['weight_score'].tolist()
        wes=wes[wes['weight_score'] >= slider7.value]
        #
        wes.T.to_csv("vis_"+content["outname"]+"_by_"+selector.value+".csv")
        #
        combined_binary_score=combined_binary_score[combined_binary_score['edge_weight_score'] >= slider7.value]
        fig3_hmp = px.imshow(wes, x=wes.columns, y=combined_binary_score.iloc[:,0:13].index, aspect='auto')
        fig3_hmp.update_layout(width=600,height=600)
        
        
        for index, row in combined_binary_score.iterrows():
            cord_x1=row['x1']
            cord_y1=row['y1']
            cord_z1=row['z1']
            cord_x2=row['x2']
            cord_y2=row['y2']
            cord_z2=row['z2']
            x = [cord_x1, cord_x2]
            y = [cord_y1, cord_y2]
            z = [cord_z1, cord_z2]
            label = row['label']
            df = pd.DataFrame({"x": x, "y":y, "z":z, "label":label})
            fig_seg = px.line_3d(df, x="x", y="y", z="z",width=600, height=600)
            fig_seg.update_traces(patch={"line": {"color": "green", "width": slider5.value, "dash":'dot'}})
            fig3_1.add_trace(fig_seg.data[0])
            
        return fig3_1, fig3_hmp, avg_score, sum_score, N

    if (selector.value == 'Weighted interaction scores-FD'):
        bb_nb_total_df=pd.merge(bb_nb_combined_score, coord, on='index1', how='left')
        bb_nb_total_df=bb_nb_total_df[bb_nb_total_df['weight_score_BB_NB'] >= slider2.value]
        bb_nb_combined_binary_score=bb_nb_combined_binary_score[bb_nb_combined_binary_score['edge_weight_score_BB_NB'] >= slider4.value]
        bb_nb_wes=bb_nb_wes[bb_nb_wes['weight_score_BB_NB'] >= slider4.value]
        avg_score=bb_nb_wes['weight_score_BB_NB'].mean()
        sum_score=bb_nb_wes['weight_score_BB_NB'].sum()
        N=bb_nb_wes.shape[0]
        
        fig4_1 = px.line_3d(coord, x="x1", y="y1", z="z1", color="chain",width=600, height=600)
        fig4_1.update_traces(showlegend = False)
        fig4_1.update_traces(line=dict(width=slider6.value))

        if (selector2.value == 'label on'):
            fig4_2 = px.scatter_3d(bb_nb_total_df, x="x1", y="y1", z="z1",color='weight_score_BB_NB',width=600, height=600,text="label")
            fig4_1.add_trace(fig4_2.data[0])
        if (selector2.value == 'label off'):
            fig4_2 = px.scatter_3d(bb_nb_total_df, x="x1", y="y1", z="z1",color='weight_score_BB_NB',width=600, height=600)
            fig4_1.add_trace(fig4_2.data[0])

        bb_nb_wes=bb_nb_wes.iloc[:,0:13]
        bb_nb_wes=(bb_nb_wes-bb_nb_wes.min())/(bb_nb_wes.max()-bb_nb_wes.min())
        bb_nb_wes=bb_nb_wes.fillna(0)

        bb_nb_combined_binary_score['edge_weight_score_BB_NB']=bb_nb_wes['weight_score_BB_NB'].tolist()
        bb_nb_wes=bb_nb_wes[bb_nb_wes['weight_score_BB_NB'] >= slider7.value]
        bb_nb_combined_binary_score=bb_nb_combined_binary_score[bb_nb_combined_binary_score['edge_weight_score_BB_NB'] >= slider7.value]
        #
        bb_nb_wes.T.to_csv("vis_"+content["outname"]+"_by_"+selector.value+".csv")
        #
        fig4_hmp = px.imshow(bb_nb_wes, x=bb_nb_wes.columns, y=bb_nb_combined_binary_score.iloc[:,0:13].index, aspect='auto')
        fig4_hmp.update_layout(width=600,height=600)

        for index, row in bb_nb_combined_binary_score.iterrows():
            cord_x1=row['x1']
            cord_y1=row['y1']
            cord_z1=row['z1']
            cord_x2=row['x2']
            cord_y2=row['y2']
            cord_z2=row['z2']
            x = [cord_x1, cord_x2]
            y = [cord_y1, cord_y2]
            z = [cord_z1, cord_z2]
            label = row['label']
            df = pd.DataFrame({"x": x, "y":y, "z":z,"label":label})
            fig_seg2 = px.line_3d(df, x="x", y="y", z="z",width=600, height=600)
            fig_seg2.update_traces(patch={"line": {"color": "green", "width": slider5.value, "dash":'dot'}})
            fig4_1.add_trace(fig_seg2.data[0])
    
        return fig4_1, fig4_hmp, avg_score, sum_score, N
    

In [32]:
def on_button3_clicked(b):
    
    df_cb_score, df_cb_binary_score=cb_score()
    df_ca_score, df_ca_binary_score=ca_score()
    df_cb_bb_nb_score, df_cb_bb_nb_binary_score=cb_bb_nb_score()
    df_ca_bb_nb_score, df_ca_bb_nb_binary_score=ca_bb_nb_score()
    df_ca_bb_b_score=ca_bb_b_score()
    df_combined_score=combine_score(df_ca_score, df_cb_score)
    df_bb_nb_combined_score=bb_nb_combine_score(df_ca_bb_nb_score, df_cb_bb_nb_score)
    df_combined_binary_score=combine_binary_score(df_ca_binary_score, df_cb_binary_score)
    df_bb_nb_combined_binary_score=bb_nb_combine_binary_score(df_ca_bb_nb_binary_score, df_cb_bb_nb_binary_score)
    df_total_score=all_score(df_combined_score, df_bb_nb_combined_score, df_ca_bb_b_score)
    df_ca_bb_b_cord=coord_from_index1()
    weighted_edge_score=combined_binary_score_with_coord(df_ca_bb_b_cord, df_combined_binary_score)
    bb_nb_weighted_edge_score=bb_nb_combined_binary_score_with_coord(df_ca_bb_b_cord, df_bb_nb_combined_binary_score)

    min=max=0
    #normalization
    if (selector3.value == '-1/1'):
        min=-1
        max=1

    if (selector3.value == '0/1'):
        min=0
        max=1

    
    df_combined_score=min_max_scale(df_combined_score, 'weight_score', min, max)
    df_combined_score['label']=df_combined_score.index
    df_bb_nb_combined_score=min_max_scale(df_bb_nb_combined_score, 'weight_score_BB_NB', min, max)
    df_bb_nb_combined_score['label']=df_bb_nb_combined_score.index
    weighted_edge_score=min_max_scale(weighted_edge_score, 'edge_weight_score', min, max)
    weighted_edge_score['label']=weighted_edge_score.index
    weighted_bb_nb_edge_score=min_max_scale(bb_nb_weighted_edge_score, 'edge_weight_score_BB_NB', min, max)
    weighted_bb_nb_edge_score['label']=weighted_bb_nb_edge_score.index
    df_combined_binary_score=min_max_scale(df_combined_binary_score, 'weight_score', min, max)
    df_bb_nb_combined_binary_score=min_max_scale(df_bb_nb_combined_binary_score, 'weight_score_BB_NB', min, max)
    

    fig,fig_hmp,avg_score, sum_score, N =plot_structuresv2(df_ca_bb_b_cord,
                          df_bb_nb_combined_score,
                          df_combined_score,
                          bb_nb_weighted_edge_score,
                          weighted_edge_score,
                          df_combined_binary_score,
                          df_bb_nb_combined_binary_score
                         )
   
    with out3:
        clear_output()
        #fig_t = make_subplots(rows=1, cols=2)
        #fig_t.add_trace(fig.data[0],row=1,col=1,)
        #fig_t.add_trace(fig_hmp.data[0],row=1,col=2,)
        display("Proffinity visualization 3D map")
        display(fig)
        display("Proffinity heatmap of critical scoring")
        display(fig_hmp)
        display("average score:"+str(avg_score))
        display("total score:"+str(sum_score))
        display("N:"+str(N))
        #display(fig_hmp)

In [33]:
button3 = widgets.Button(description="Visualization")
button3.on_click(on_button3_clicked)
slider = widgets.VBox([slider1,slider2,slider3,slider4,slider5,slider7,slider6])
#ui = widgets.HBox([button3,selector,slider,selector2,selector3])
#display(ui)

In [34]:
scene = HBox([VBox([out,button,out2,button2,HBox([selector,selector2,selector3]),slider]),VBox([out3,button3])])
display(scene)

### Instruction
step1. Load FI params: load feature importance parameters based on the selected ML model for WRS and WIS
<br>
step2. Load Inputs: load a single PDB complex in the "model" folder to be scored
<br>
step3. Select scoring options:
<br>
- residue scores: WRS, non-bonded scores: WIS; 
- label on/off: residue labels on figure
- -1/1 or 0/1: mode of score normalization
##### Note: if switch to 0/1, plesae adjust the threshold accordingly (default:-1/1)

step4. Adjust threshold option
<br>
- WRS-IF: score cutoff for weighted residue scores (interface)
- WS-FD: score cutoff for weighted residue scores (fold)
- WIS-IF: score cutoff for weighted interaction scores (interface)
- WIS-FD: score cutoff for weighted interaction scores (fold)
<br>
##### Note: score cutoff need to be adjusted according to the mode of score normalization
step5. Adjust asthetic option
<br>
- edge width: width of the non-bonded edge on the 3D figure
- edge thres: score cutoff for the edge shown on the heatmap 
- chain width: width of the chain backbone on the 3D figure 
